In [1]:
import requests
import json
import prettytable

Note: `prettytable` is a Python package which may need to be instlled: `pip install prettytable`   
> *PrettyTable is a simple Python library designed to make it quick and easy to represent tabular data in visually appealing ASCII tables.*   

https://pypi.python.org/pypi/PrettyTable

In [58]:
# an array of strings which represent BLS Series IDs
seriesID = ['LAUMT423798000000003']
startYear = 2016
endYear = 2017

In [59]:
seriesID

['LAUMT423798000000003']

In [60]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": seriesID,"startyear":"{}".format(startYear), "endyear":"{}".format(endYear)})

In [61]:
data

'{"startyear": "2016", "endyear": "2017", "seriesid": ["LAUMT423798000000003"]}'

In [62]:
# get the data using API
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)

In [63]:
p

<Response [200]>

In [64]:
json_data = json.loads(p.text)

In [52]:
json_data

{'Results': {'series': [{'data': [{'footnotes': [{}],
      'period': 'M12',
      'periodName': 'December',
      'value': '4.3',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M11',
      'periodName': 'November',
      'value': '4.5',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M10',
      'periodName': 'October',
      'value': '5.0',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M09',
      'periodName': 'September',
      'value': '5.1',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M08',
      'periodName': 'August',
      'value': '5.5',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M07',
      'periodName': 'July',
      'value': '5.7',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M06',
      'periodName': 'June',
      'value': '5.4',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'M05',
      'periodName': 'May',
      'value': '5.0',
      'ye

In [65]:
# extract the response into a PrettyTable
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])



In [66]:
x

In [67]:
print(x)

+----------------------+------+--------+-------+--------------+
|      series id       | year | period | value |  footnotes   |
+----------------------+------+--------+-------+--------------+
| LAUMT423798000000003 | 2017 |  M08   |  5.1  | Preliminary. |
| LAUMT423798000000003 | 2017 |  M07   |  5.2  |              |
| LAUMT423798000000003 | 2017 |  M06   |  4.8  |              |
| LAUMT423798000000003 | 2017 |  M05   |  4.9  |              |
| LAUMT423798000000003 | 2017 |  M04   |  4.4  |              |
| LAUMT423798000000003 | 2017 |  M03   |  4.6  |              |
| LAUMT423798000000003 | 2017 |  M02   |  4.9  |              |
| LAUMT423798000000003 | 2017 |  M01   |  4.8  |              |
| LAUMT423798000000003 | 2016 |  M12   |  4.3  |              |
| LAUMT423798000000003 | 2016 |  M11   |  4.5  |              |
| LAUMT423798000000003 | 2016 |  M10   |  5.0  |              |
| LAUMT423798000000003 | 2016 |  M09   |  5.1  |              |
| LAUMT423798000000003 | 2016 |  M08   |